# Лабораторна робота 3 з Теоретико-числових алгоритмів в криптології
## Тема: Реалiзацiя та застосування алгоритму дискретного логарифмування **index-calculus**

**Варіант:** 2\
**Виконали:** Бондар Петро, Кістаєв Матвій\
**Група:** ФІ-03

### Посилання
Github: https://github.com/Pechenkya/NTA-Labs-Bondar-Kistaiev-FI-03/tree/main/Lab-3

Docker image: 

In [230]:
import numpy as np
from math import exp, sqrt, log, ceil, gcd
import random
# Use factorization from Lab 1
from factor_module import general_factor, check_prime

base = 2

## Генеруємо факторну базу

In [231]:
def gen_factor_base(n):
    c = 3.38 # recommended constant
    B_lim = int(c * exp(0.5 * sqrt(log(n, base) * log(log(n, base), base))))
    
    S = []
    for a in range(2, B_lim):
        if check_prime(a):
            S.append(a)
    
    return S

## Генеруємо систему порівнянь 

In [232]:
C = 15

def is_smooth(val, S):
    D = [0] * len(S)
    for i in range(len(S)):
        while val % S[i] == 0:
            D[i] += 1
            val //= S[i]
        
    if val != 1:
        return [False, []]
    else:
        return [True, D]


def gen_equations(a, n, S, number_of_eq=0):
    if number_of_eq == 0:
        number_of_eq = len(S) + C

    A = []
    b = []

    curr_power = 1
    curr_val = a

    while len(b) < number_of_eq:
        [smooth, pows] = is_smooth(curr_val, S)
        if smooth:
            A.append(pows)
            b.append(curr_power)

        curr_val = (curr_val * a) % n
        curr_power += 1

        if curr_val == 1:
            break
    
    print(f"Generated equations: {len(b)}")
    return (np.array(A), np.array(b))

In [ ]:
from multiprocessing import Pool

def append_equation():

def gen_equations_parallel(a, n, S, number_of_eq=0):
    if number_of_eq == 0:
        number_of_eq = len(S) + C

    A = []
    b = []

    curr_power = 1
    curr_val = a

    with Pool() as pool:
      result = pool.map(is_smooth, pow(2, [], n))

## Розв'язування системи лінійних порівнянь за модулем

In [233]:
def solve_modular_eq(A_in, b, m):
    # Create system of Linear Diophante eq by appending new unknowns
    M = m * np.identity(len(A_in), dtype=int)
    A = np.concatenate([A_in, M], axis=1)
    print(A)

## Знаходимо відповідний $\log_\alpha \beta$

In [234]:
def find_index_sequential_comparing(a, beta, n, S, A, b):
    curr_ind = 0
    curr_val = beta

    A_list = A.tolist()
    for _ in range(n-1):
        [smooth, pows] = is_smooth(curr_val, S)
        if smooth and A_list.count(pows) != 0:
            corr_a_ind = b[A_list.index(pows)]
            return (corr_a_ind - curr_ind) % (n - 1)
        
        curr_ind += 1
        curr_val = (curr_val * a) % n

    raise RuntimeError("Can't find index!")

def find_index(a, beta, n, S, S_idxs):
    curr_ind = 0
    curr_val = beta

    for _ in range(n-1):
        [smooth, pows] = is_smooth(curr_val, S)
        if smooth:
            corr_a_ind = np.dot(S_idxs, pows)
            return (corr_a_ind - curr_ind) % (n - 1)
        
        curr_ind += 1
        curr_val = (curr_val * a) % n

    raise RuntimeError("Can't find index!")



In [240]:
def solve_brute(alpha, beta, n):
    Base = gen_factor_base(n)
    A, b = gen_equations(alpha, n, Base)
    res = find_index_sequential_comparing(alpha, beta, n, Base, A, b)
    return res

solve_brute(36237829, 46148350, 77875409)

Generated equations: 165


36773395